# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
from citipy import citipy
from random import uniform
from scipy import stats as st
import datetime
import matplotlib.pyplot as plt
import requests
import json
from pathlib import Path

# Import API keys
from api_keys import (weather_api_key, g_key)

## Function Block

In [ ]:
def randLatLong():
    Lat = uniform(-90,90)
    Lng = uniform(-180,180)
    return Lat, Lng

## Generate Cities List

In [ ]:
numCities = 750
cityList = []
for x in range(numCities):
    cityList.append(randLatLong())

cityName = []
cityCountry = []
for city in cityList:
    lat = city[0]
    lng = city[1]
    city = citipy.nearest_city(lat, lng)
    cityName.append(city.city_name)
    cityCountry.append(city.country_code)
city_df = pd.DataFrame(data = cityList)
city_df['CityName'] = cityName
city_df['CountryCode'] = cityCountry
city_df.columns = ['Lat Guess', 'Long Guess', 'CityName','CountryCode']
city_df['Temperature'] = ''
city_df['Humidity'] = ''
city_df['Cloudiness'] = ''
city_df['Wind Speed'] = ''
city_df['Lat Real'] = ''
city_df['Long Real'] = ''
city_df['Datetime'] = ''
city_df.head()

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [ ]:
# Outputting sample query to be able to look at json format
qry = query_url + city_df['CityName'][0] + ',' + city_df['CountryCode'][0]
print(qry)

In [ ]:
lat = []
lon = []
temp = []
hum = []
cloud = []
wind = []
date = []
for index,row in city_df.iterrows():
    print(f'Working on City Number {index}: {row[2]}, {row[3]}')
    try:
        response = requests.get(query_url + row[2] + ',' + row[3]).json()
        city_df['Lat Real'][index] = response['coord']['lat']
        city_df['Long Real'][index] = response['coord']['lon']
        city_df['Temperature'][index] = response['main']['temp_max']
        city_df['Humidity'][index] = response['main']['humidity']
        city_df['Cloudiness'][index] = response['clouds']['all']
        city_df['Wind Speed'][index] = response['wind']['speed']
        city_df['Datetime'][index] = response['dt']
    except:
        city_df['Lat Real'][index] = np.nan
        city_df['Long Real'][index] = np.nan
        city_df['Temperature'][index] = np.nan
        city_df['Humidity'][index] = np.nan
        city_df['Cloudiness'][index] = np.nan
        city_df['Wind Speed'][index] = np.nan
        city_df['Datetime'][index] = np.nan
        print('')
        print(f'City Number is missing data')
        print('')

In [ ]:
cols = ['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Lat Real', 'Long Real', 'Datetime']
city_df[cols].replace('', np.nan, inplace=True)
city_df.dropna(how='any', inplace=True)
city_df.head(100)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_df[cols] = city_df[cols].astype(float)
city_df.describe()

In [ ]:
printpath = Path.cwd() / 'cities.csv'
city_df.to_csv(printpath, index=False)

In [ ]:
parentpath = Path(Path.cwd()).parents[0] / 'VacationPy' / 'cities.csv'
city_df.to_csv(parentpath, index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# We assume that all data was pulled on the same date, so just grab one datetime
day = datetime.datetime.fromtimestamp(int(city_df['Datetime'][0])).strftime('%Y-%m-%d')

In [ ]:
scat1 = plt.scatter(city_df['Lat Real'], city_df['Temperature'])
scat1 = plt.xlabel('Latitude')
scat1 = plt.ylabel('Temperature (F)')
scat1 = plt.title(f"Temperature vs Latitude on {day}")
scat1 = plt.grid(True)
plt.savefig(f"Temperature Vs Latitude {day}.png")

## Latitude vs. Humidity Plot

In [ ]:
scat2 = plt.scatter(city_df['Lat Real'], city_df['Humidity'])
scat2 = plt.xlabel('Latitude')
scat2 = plt.ylabel('Humidity (%)')
scat2 = plt.title(f"Humidity vs Latitude on {day}")
scat2 = plt.grid(True)
plt.savefig(f"Humidity Vs Latitude {day}.png")

## Latitude vs. Cloudiness Plot

In [ ]:
scat3 = plt.scatter(city_df['Lat Real'], city_df['Cloudiness'])
scat3 = plt.xlabel('Latitude')
scat3 = plt.ylabel('Cloudiness (%)')
scat3 = plt.title(f"Cloudiness vs Latitude on {day}")
scat3 = plt.grid(True)
plt.savefig(f"Cloudiness Vs Latitude {day}.png")

## Latitude vs. Wind Speed Plot

In [ ]:
scat4 = plt.scatter(city_df['Lat Real'], city_df['Wind Speed'])
scat4 = plt.xlabel('Latitude')
scat4 = plt.ylabel('Wind Speed (mph)')
scat4 = plt.title(f"Wind Speed vs Latitude on {day}")
scat4 = plt.grid(True)
plt.savefig(f"Wind Speed Vs Latitude {day}.png")

## Linear Regression

In [ ]:
northCities = city_df.loc[city_df['Lat Real'] > 0]
southCities = city_df.loc[city_df['Lat Real'] < 0]

In [ ]:
northCities.describe()

In [ ]:
southCities.describe()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(northCities['Lat Real'], northCities['Temperature'])
print(f'The r-value is {rvalue**2:0.4f}')
scat5 = plt.scatter(northCities['Lat Real'], northCities['Temperature'])
scat5 = plt.xlabel('Latitude')
scat5 = plt.ylabel('Temperature (F)')
scat5 = plt.title(f"Northern Hemisphere Temperature vs Latitude on {day}")
scat5 = plt.grid(True)
lineq5 = slope * northCities['Lat Real'] + intercept
plt.plot(northCities['Lat Real'], lineq5, color='red')
plt.savefig(f"Northern Hemisphere Temperature Vs Latitude {day}.png")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(southCities['Lat Real'], southCities['Temperature'])
print(f'The r-value is {rvalue:0.4f}')
scat6 = plt.scatter(southCities['Lat Real'], southCities['Temperature'])
scat6 = plt.xlabel('Latitude')
scat6 = plt.ylabel('Temperature (F)')
scat6 = plt.title(f"Southern Hemisphere Temperature vs Latitude on {day}")
scat6 = plt.grid(True)
lineq6 = slope * southCities['Lat Real'] + intercept
plt.plot(southCities['Lat Real'], lineq6, color='red')
plt.savefig(f"Southern Hemisphere Temperature Vs Latitude {day}.png")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(northCities['Lat Real'], northCities['Humidity'])
print(f'The r-value is {rvalue:0.4f}')
scat7 = plt.scatter(northCities['Lat Real'], northCities['Humidity'])
scat7 = plt.xlabel('Latitude')
scat7 = plt.ylabel('Humidity (%)')
scat7 = plt.title(f"Northern Hemisphere Humidity vs Latitude on {day}")
scat7 = plt.grid(True)
lineq7 = slope * northCities['Lat Real'] + intercept
plt.plot(northCities['Lat Real'], lineq7, color='red')
plt.savefig(f"Northern Hemisphere Humidity Vs Latitude {day}.png")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(southCities['Lat Real'], southCities['Humidity'])
print(f'The r-value is {rvalue:0.4f}')
scat8 = plt.scatter(southCities['Lat Real'], southCities['Humidity'])
scat8 = plt.xlabel('Latitude')
scat8 = plt.ylabel('Humidity (%)')
scat8 = plt.title(f"Southern Hemisphere Humidity vs Latitude on {day}")
scat8 = plt.grid(True)
lineq8 = slope * southCities['Lat Real'] + intercept
plt.plot(southCities['Lat Real'], lineq8, color='red')
plt.savefig(f"Southern Hemisphere Humidity Vs Latitude {day}.png")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(northCities['Lat Real'], northCities['Cloudiness'])
print(f'The r-value is {rvalue:0.4f}')
scat9 = plt.scatter(northCities['Lat Real'], northCities['Cloudiness'])
scat9 = plt.xlabel('Latitude')
scat9 = plt.ylabel('Cloudiness (%)')
scat9 = plt.title(f"Northern Hemisphere Cloudiness vs Latitude on {day}")
scat9 = plt.grid(True)
lineq9 = slope * northCities['Lat Real'] + intercept
plt.plot(northCities['Lat Real'], lineq9, color='red')
plt.savefig(f"Northern Hemisphere Cloudiness Vs Latitude {day}.png")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(southCities['Lat Real'], southCities['Cloudiness'])
print(f'The r-value is {rvalue:0.4f}')
scat10 = plt.scatter(southCities['Lat Real'], southCities['Cloudiness'])
scat10 = plt.xlabel('Latitude')
scat10 = plt.ylabel('Cloudiness (%)')
scat10 = plt.title(f"Southern Hemisphere Cloudiness vs Latitude on {day}")
scat10 = plt.grid(True)
lineq10 = slope * southCities['Lat Real'] + intercept
plt.plot(southCities['Lat Real'], lineq10, color='red')
plt.savefig(f"Southern Hemisphere Cloudiness Vs Latitude {day}.png")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(northCities['Lat Real'], northCities['Wind Speed'])
print(f'The r-value is {rvalue:0.4f}')
scat11 = plt.scatter(northCities['Lat Real'], northCities['Wind Speed'])
scat11 = plt.xlabel('Latitude')
scat11 = plt.ylabel('Wind Speed (mph)')
scat11 = plt.title(f"northern Hemisphere Wind Speed vs Latitude on {day}")
scat11 = plt.grid(True)
lineq11 = slope * northCities['Lat Real'] + intercept
plt.plot(northCities['Lat Real'], lineq11, color='red')
plt.savefig(f"Northern Hemisphere Wind Speed Vs Latitude {day}.png")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
slope, intercept, rvalue, pvalue, stderr = st.linregress(southCities['Lat Real'], southCities['Wind Speed'])
print(f'The r-value is {rvalue:0.4f}')
scat9 = plt.scatter(southCities['Lat Real'], southCities['Wind Speed'])
scat9 = plt.xlabel('Latitude')
scat9 = plt.ylabel('Wind Speed (%)')
scat9 = plt.title(f"Southern Hemisphere Wind Speed vs Latitude on {day}")
scat9 = plt.grid(True)
lineq9 = slope * southCities['Lat Real'] + intercept
plt.plot(southCities['Lat Real'], lineq9, color='red')
plt.savefig(f"Southern Hemisphere Wind Speed Vs Latitude {day}.png")